In [1]:
%pip install ultralytics > /dev/null

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ultralytics
import torch
import os
import json
import yaml
import pandas as pd
import shutil

In [3]:
train = pd.read_csv('/kaggle/input/bird-genus-multi-class-image-classification/train.csv')
test = pd.read_csv('/kaggle/input/bird-genus-multi-class-image-classification/test.csv')

classes = train['genus'].unique()

In [4]:
os.mkdir('/kaggle/working/dataset')
os.mkdir('/kaggle/working/dataset/train')
os.mkdir('/kaggle/working/dataset/test')

for cls in classes:
    os.mkdir(f'/kaggle/working/dataset/train/{cls}')
    os.mkdir(f'/kaggle/working/dataset/test/{cls}')

In [5]:
fromDir = '/kaggle/input/bird-genus-multi-class-image-classification/images/train/'

for row in train.itertuples(): # idx2=filename, idx3=genus
    shutil.copyfile(fromDir + row[2], f'/kaggle/working/dataset/train/{row[3]}/{row[2]}')
    shutil.copyfile(fromDir + row[2], f'/kaggle/working/dataset/test/{row[3]}/{row[2]}')

In [6]:
# total = 0
# for dir in os.listdir('/kaggle/working/dataset'):
#     total += len(os.listdir(f'/kaggle/working/dataset/{dir}'))
#     print(len(os.listdir(f'/kaggle/working/dataset/{dir}')))

In [7]:
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available, using CPU instead")
    device = torch.device("cpu")

GPU is available


In [8]:
model = ultralytics.YOLO('yolov8x-cls.pt')

model.to(device)

100%|██████████| 110M/110M [00:00<00:00, 257MB/s] 


YOLO(
  (model): ClassificationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 80, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(160, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(400, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_

In [9]:
# Currently not using wandb
os.environ['WANDB_DISABLED'] = 'true'

In [10]:
model.train(data='/kaggle/working/dataset', epochs=20, imgsz=300)

engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=/kaggle/working/dataset, epochs=20, time=None, patience=100, batch=16, imgsz=300, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8

2024-05-07 18:55:26,287	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-07 18:55:27,115	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-05-07 18:55:29.504867: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 18:55:29.505018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 18:55:29.639024: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLA

Overriding model.yaml nc=1000 with nc=27

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   7375360  ultralyt

wandb: Tracking run with wandb version 0.16.6
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...


...

                   all      0.999          1

20 epochs completed in 0.475 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 112.5MB
Optimizer stripped from runs/classify/train/weights/best.pt, 112.5MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.2.10 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
YOLOv8x-cls summary (fused): 133 layers, 56157787 parameters, 0 gradients, 153.8 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /kaggle/working/dataset/train... found 4258 images in 27 classes ✅ 
val: None...
test: /kaggle/working/dataset/test... found 4258 images in 27 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 134/134 [00:19<00:00,  6.88it/s]


                   all      0.999          1
Speed: 0.1ms preprocess, 4.1ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                  lr/pg0 ▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
wandb:                  lr/pg1 ▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
wandb:                  lr/pg2 ▃▆██▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁▁
wandb:   metrics/accuracy_top1 ▁▄▅▅▆▆▇▇▇███████████
wandb:   metrics/accuracy_top5 ▁▆▆▇████████████████
wandb:            model/GFLOPs ▁
wandb:        model/parameters ▁
wandb: model/speed_PyTorch(ms) ▁
wandb:              train/loss █▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
wandb:                val/loss █▆▅▅▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:                  lr/pg0 4e-05
wandb:                  lr/pg1 4e-05
wandb:                  lr/pg2 4e-05
wandb:   metrics/accuracy_top1 0.99906
wandb:   metrics/accuracy_top5 1.0
wandb:            model/GFLOPs 154.338
wandb:        model/parameters 56176427
wandb: model/speed_PyTorch(ms) 4.013
wandb:              train/loss 0.16145
wandb:                val/loss 2.36312
wandb: 
wandb: 

ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7fd907a69a80>
curves: []
curves_results: []
fitness: 0.9995302855968475
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9990605711936951, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9995302855968475}
save_dir: PosixPath('runs/classify/train')
speed: {'preprocess': 0.1389826335163394, 'inference': 4.12619477877207, 'loss': 0.0006593186399532406, 'postprocess': 0.0005257751192493358}
task: 'classify'
top1: 0.9990605711936951
top5: 1.0

In [11]:
os.mkdir('/kaggle/working/predimg')

In [12]:
fromDir = '/kaggle/input/bird-genus-multi-class-image-classification/images/test/'
for img in os.listdir(fromDir):
    shutil.copy(fromDir + img, f'/kaggle/working/predimg/{img}')

In [13]:
predimgs = [f'/kaggle/working/predimg/{file}' for file in os.listdir('/kaggle/working/predimg')]

In [14]:
ids = []
genus = []

for row in test.itertuples():
    ids.append(row[1])
    filename = row[2]
    prediction = model(f'/kaggle/working/predimg/{filename}')
    genus.append(prediction[0].names[int(prediction[0].probs.data.argmax().cpu().numpy())])


image 1/1 /kaggle/working/predimg/393ae222-30df-4382-853d-189ca54d3ce7.jpg: 320x320 Strigidae 1.00, Accipitridae 0.00, Falconidae 0.00, Phasianidae 0.00, Emberizidae 0.00, 9.9ms
Speed: 9.5ms preprocess, 9.9ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /kaggle/working/predimg/d5c0ea48-9f86-40bd-ab8c-4844e442d946.jpg: 320x320 Laniidae 0.99, Hirundinidae 0.01, Paridae 0.00, Muscicapidae 0.00, Laridae 0.00, 9.9ms
Speed: 4.8ms preprocess, 9.9ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /kaggle/working/predimg/73909c37-cdcc-423b-aee2-a7e531e38c58.jpg: 320x320 Passeridae 0.41, Laridae 0.27, Hirundinidae 0.15, Paridae 0.14, Laniidae 0.01, 9.9ms
Speed: 4.6ms preprocess, 9.9ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /kaggle/working/predimg/87abdfb0-ded0-4732-ac49-ba0c219265a9.jpg: 320x320 Turdidae 1.00, Phasianidae 0.00, Muscicapidae 0.00, Motacillidae 0.00, Emberizidae 0.00, 9.8ms
Speed: 4.5m

In [15]:
pred_df = pd.DataFrame({'id': ids, 'genus': genus})

In [16]:
pred_df.to_csv('bird_submission.csv', index=False)